<p style="float: left;"><a href="upper-type-bounds.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="lower-type-bounds.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>

# Variance annotations

**Variance is how subtyping between more complex types (generic types) relates to subtyping between their components (type variables).**

- Suppose `A` and `B` are types, and `K<U>` denotes application of a type constructor `K` with type variable `U`. 

- The kind of subtyping relationships are:

    - **variant** if covariant or contravariant.
        
        - **covariant** if `A` $\leq$ `B`, then `K<A>` $\leq$ `K<B>`.
          
        - **contravariant** if `A` $\leq$ `B`, then `K<B>` $\leq$ `K<A>`.
          
    - **invariant** or nonvariant if not variant.

- Scala class syntax for the different subtyping relationships:

    ```scala
    class Foo[+A] // A covariant class
    class Bar[-A] // A contravariant class
    class Baz[A]  // An invariant class
    
    ```

## The variance problem

- The combination of type parameters and subtyping poses some interesting questions. 
 
    - _Should **Stack[String]** be a subtype of **Stack[AnyRef]**?_

        - This seems OK, since a stack of `String` is a special case of a stack of `AnyRef`.
    
        <br/>
    
- **In a purely functional world, all types could be covariant**. However, <span style="color:red">**the situation changes once we introduce mutable data.**</span>

    ```java
    class Fruit
    
    val x = new Array[String](1)
    val y: Array[AnyRef] = x // Covariant assignment, this would be legit in Java not in Scala!!!
    y(0) = new Fruit         // Introducing a `Fruit` into `Array[String]`?
    ```

    <br/>

    - In Java, arrays are covariant. <span style="color:black">**This might seem natural but leads to safety problems that require special _runtime checks_.**</span>
    
        - **<span style="color:red">We would end up storing a `Fruit` in an array `Array[String]` which clearly violates type soundness.</span>**

## Scala's variance approuch

- Scala solves the variance problem statically instead of in runtime.

- Covariant type variables may appear in:
    
    1. The types of values (fields) defined in the class.
       
       ```scala
       class Box[+A](val value: A) // covariant in A
       
       ```

        <br/>
       
    3. The result types of methods in the class.
       
        ```scala
        trait Producer[+A]
          def produce(): A // result type is covariant
          
        ```
        
    5. Type variables to other covariant types.
       
       ```scala
       case class Node[+A](head: A, tail: List[A]) // List is covariant
       
       ```

    <br/>
    
- **But, <span style="color:red">formal parameter types of methods cannot be covariant**.<span>

    
    ```scala
    class Array[+A] {
      def apply(index: Int): A
      def update(index: Int, elem: A) // ❌ not allowed
    }
    ```

    <br/>

    - **Since the `update` method might change the internal state (mutation):**
      
        -  <span style="color:red">**It might invalidate the subtyping relationship (break type safety).**</span>

        <br/>

    - **There are methods which do not mutate state, but type parameter still appears contravariant position...**

        ```scala
        class Stack[+A] {
            def push(x: A): Stack[A] = . . . // Do not mutate the internal state, returns a new stack
        }
        ```

        <br/>

         - **Stacks are purely functional data structures.** <span style="color:red">**Covariant subtyping should be allowed**.</span>
       
         - For solving this problem we can use [lower type parameter bound](lower-type-bounds.ipynb).

<p style="float: left;"><a href="upper-type-bounds.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="lower-type-bounds.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>